In [ ]:
# run_all.py
# Advanced Time Series Forecasting with Neural State Space Models (NSSMs)
# This script generates data, trains a VAE-style NSSM and a baseline LSTM, and evaluates them.
# Requirements: Python 3.8+, torch, numpy, matplotlib, scikit-learn
# Install: pip install torch numpy matplotlib scikit-learn

import os
import math
import json
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTDIR = Path("outputs")
OUTDIR.mkdir(exist_ok=True)

# -------------------- Dataset generation --------------------
def generate_multivariate_series(seq_len=1000, n_features=5, seed=0):
    np.random.seed(seed)
    t = np.arange(seq_len)
    data = np.zeros((seq_len, n_features), dtype=float)
    for f in range(n_features):
        trend = 0.001 * (f + 1) * t
        season = 0.5 * np.sin(2 * np.pi * (t / (50 + 10 * f)) + (f * 0.5))
        ar = np.zeros(seq_len)
        phi = 0.6 - 0.05 * f
        for i in range(1, seq_len):
            ar[i] = phi * ar[i - 1] + 0.05 * np.random.randn()
        noise = 0.2 * np.random.randn(seq_len)
        data[:, f] = trend + season + ar + noise
    return data

# Sliding-window dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_len=50, pred_len=10):
        self.data = data.astype(np.float32)
        self.input_len = input_len
        self.pred_len = pred_len
        self.seq_len, self.n_features = self.data.shape
        self.indices = []
        for i in range(0, self.seq_len - input_len - pred_len + 1):
            self.indices.append(i)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        i = self.indices[idx]
        x = self.data[i:i + self.input_len]
        y = self.data[i + self.input_len:i + self.input_len + self.pred_len]
        return x, y

# -------------------- NSSM (VAE-style amortized inference) --------------------
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim, n_layers=1):
        super().__init__()
        # bidirectional LSTM encoder
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, bidirectional=True)
        self.fc_mean = nn.Linear(hidden_dim * 2, z_dim)
        self.fc_logvar = nn.Linear(hidden_dim * 2, z_dim)
    def forward(self, x):
        # x: (B, T, D)
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        mean = self.fc_mean(h)
        logvar = self.fc_logvar(h)
        return mean, logvar

class TransitionNet(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, z_dim)
        )
    def forward(self, z):
        return self.net(z)

class ObservationNet(nn.Module):
    def __init__(self, z_dim, out_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, z):
        return self.net(z)

class NSSM(nn.Module):
    def __init__(self, input_dim, z_dim=8, enc_h=64, trans_h=64, obs_h=64):
        super().__init__()
        self.encoder = Encoder(input_dim, enc_h, z_dim)
        self.transition = TransitionNet(z_dim, trans_h)
        self.observation = ObservationNet(z_dim, input_dim, obs_h)
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    def forward(self, x_context, pred_len=10):
        # x_context: (B, T, D)
        mu, logvar = self.encoder(x_context)
        z = self.reparameterize(mu, logvar)  # initial latent
        preds = []
        for t in range(pred_len):
            dz = self.transition(z)
            z = z + dz  # residual update
            y_hat = self.observation(z)
            preds.append(y_hat.unsqueeze(1))
        preds = torch.cat(preds, dim=1)
        return preds, mu, logvar

# -------------------- Baseline LSTM predictor --------------------
class LSTMPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, pred_len=10):
        super().__init__()
        self.pred_len = pred_len
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, input_dim * pred_len)
    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        out = self.fc(h)
        out = out.view(x.size(0), self.pred_len, -1)
        return out

# -------------------- Training utilities --------------------
def loss_nssm(preds, target, mu, logvar, beta=1e-3):
    rec_loss = ((preds - target) ** 2).mean()
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return rec_loss + beta * kl, rec_loss.item(), kl.item()

def train_one_epoch_nssm(model, loader, opt):
    model.train()
    total_loss = 0.0
    total_rec = 0.0
    total_kl = 0.0
    for x, y in loader:
        x = x.to(device); y = y.to(device)
        opt.zero_grad()
        preds, mu, logvar = model(x, pred_len=y.size(1))
        loss, rec, kl = loss_nssm(preds, y, mu, logvar)
        loss.backward()
        opt.step()
        total_loss += loss.item() * x.size(0)
        total_rec += rec * x.size(0)
        total_kl += kl * x.size(0)
    n = len(loader.dataset)
    return total_loss / n, total_rec / n, total_kl / n

def train_one_epoch_lstm(model, loader, opt, criterion):
    model.train()
    total_loss = 0.0
    for x, y in loader:
        x = x.to(device); y = y.to(device)
        opt.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        opt.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

def evaluate_model(model, loader, model_type='nssm'):
    model.eval()
    Ys = []
    Yh = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device); y = y.to(device)
            if model_type == 'nssm':
                preds, _, _ = model(x, pred_len=y.size(1))
            else:
                preds = model(x)
            Ys.append(y.cpu().numpy())
            Yh.append(preds.cpu().numpy())
    Ys = np.concatenate(Ys, axis=0)
    Yh = np.concatenate(Yh, axis=0)
    rmse = math.sqrt(mean_squared_error(Ys.reshape(-1, Ys.shape[-1]), Yh.reshape(-1, Yh.shape[-1])))
    mae = mean_absolute_error(Ys.reshape(-1, Ys.shape[-1]), Yh.reshape(-1, Yh.shape[-1]))
    return rmse, mae, Ys, Yh

# -------------------- Main run --------------------
def main():
    # Hyperparameters
    seq_len = 1000
    n_features = 5
    input_len = 50
    pred_len = 10
    batch_size = 32
    z_dim = 8
    epochs = 12  # increase for better results

    data = generate_multivariate_series(seq_len=seq_len, n_features=n_features)
    np.savetxt(OUTDIR / 'sample_data.csv', data, delimiter=',')

    # Train/test split by time
    train_ratio = 0.8
    split = int(seq_len * train_ratio)
    train_series = data[:split]
    test_series = data[split - input_len:]  # keep context for windows

    train_ds = TimeSeriesDataset(train_series, input_len=input_len, pred_len=pred_len)
    test_ds = TimeSeriesDataset(test_series, input_len=input_len, pred_len=pred_len)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    # NSSM
    nssm = NSSM(input_dim=n_features, z_dim=z_dim).to(device)
    opt_n = torch.optim.Adam(nssm.parameters(), lr=1e-3)
    for ep in range(epochs):
        loss, rec, kl = train_one_epoch_nssm(nssm, train_loader, opt_n)
        print(f"[NSSM] Epoch {ep+1}/{epochs}: loss={loss:.6f} rec={rec:.6f} kl={kl:.6f}")
    torch.save(nssm.state_dict(), OUTDIR / 'nssm.pth')

    # Baseline LSTM
    lstm = LSTMPredictor(input_dim=n_features, hidden_dim=64, pred_len=pred_len).to(device)
    opt_l = torch.optim.Adam(lstm.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    for ep in range(epochs):
        loss_l = train_one_epoch_lstm(lstm, train_loader, opt_l, criterion)
        print(f"[LSTM] Epoch {ep+1}/{epochs}: train_mse={loss_l:.6f}")
    torch.save(lstm.state_dict(), OUTDIR / 'lstm.pth')

    # Evaluate
    nssm_rmse, nssm_mae, Ys_n, Yh_n = evaluate_model(nssm, test_loader, model_type='nssm')
    lstm_rmse, lstm_mae, Ys_l, Yh_l = evaluate_model(lstm, test_loader, model_type='lstm')
    print("\nEvaluation on test set:")
    print(f"NSSM RMSE={nssm_rmse:.6f}, MAE={nssm_mae:.6f}")
    print(f"LSTM RMSE={lstm_rmse:.6f}, MAE={lstm_mae:.6f}")

    results = {
        'nssm_rmse': float(nssm_rmse),
        'nssm_mae': float(nssm_mae),
        'lstm_rmse': float(lstm_rmse),
        'lstm_mae': float(lstm_mae)
    }
    (OUTDIR / 'results.json').write_text(json.dumps(results, indent=2))

    # Quick plot comparing first test sample predictions (feature 0)
    plt.figure(figsize=(8, 4.5))
    gt = Ys_n[0, :, 0]
    p_n = Yh_n[0, :, 0]
    p_l = Yh_l[0, :, 0]
    T = np.arange(len(gt))
    plt.plot(T, gt, label='Ground truth')
    plt.plot(T, p_n, label='NSSM pred')
    plt.plot(T, p_l, label='LSTM pred')
    plt.legend()
    plt.title('First test sample - feature 0 predictions')
    plt.tight_layout()
    plt.savefig(OUTDIR / 'pred_compare.png', dpi=150)
    print(f"Outputs saved to {OUTDIR.resolve()}")

if __name__ == '__main__':
    main()